# MolShapeGAN: LAMMPS simulator

## Generative multiscale analysis of de novo proteome-inspired molecular structures and nanomechanical optimization using a VoxelPerceiver transformer model

Zhenze Yang, Yu-Chuan Hsu, Markus J. Buehler, "Generative multiscale analysis of de novo proteome-inspired molecular structures and nanomechanical optimization using a VoxelPerceiver transformer model," Journal of the Mechanics and Physics of Solids, Volume 170, January 2023, 105098.

mbuehler@MIT.EDU

In [ ]:
from math import *
from random import random
import os

import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits import mplot3d 
plt.style.use('seaborn-whitegrid')
import numpy as np
import cv2
from PIL import Image
import shutil

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import csv

import math
import shutil
import statistics          
import re

In [ ]:
import numpy as np
import os
import trimesh  
import time, warnings
from PIL import Image
import glob
import matplotlib.pyplot as plt
import cv2
        
def img2vox(loc, end='png', thresh=127, im_sh=False):
    vox = []
    imgs = sorted(glob.glob(loc+'/*.'+end), key=lambda x: (len(x), x))

    print('found',len(imgs),'images.',imgs)
    for i in imgs:
        new_frame = Image.open(i).convert('RGB').convert('L')
        
        new_frame =np.array(new_frame)
        _, new_frame = cv2.threshold(new_frame,thresh,255,cv2.THRESH_BINARY)
        
        if im_sh:
                plt.imshow(new_frame, interpolation='nearest',cmap="hot")
                plt.colorbar()
                plt.show()
                
        vox.append(np.array(new_frame))

    return np.array(vox)

def TwoDimg23Dvox(img_,maxheight=10, invvv=False, thresh=0, 
                  normalize=False,norm_zero=False,clipvalue=0,sat=1,GaussSmoothimage=False, iblur=0, BilatSmoothimage=False, centerrep=0, darea=0, invertresult=False):

   vox = []
   print ("Number of images: ", len (img_[:]))  
   for imc in range (len(img_[:])):
    img=img_[imc]
    print ("Considering image: ", imc)
    if GaussSmoothimage==True:
        print ("smoothen using Gaussian Blur...")
        #Applying the blur filter
        img = cv2.GaussianBlur(img,(5,5),0)
        #img = cv2.bilateralFilter(img,9,75,75) 
        
        for ij in range (iblur):
                img = cv2.GaussianBlur(img, (3,3), 10,10)
    if BilatSmoothimage==True:
        print ("smoothen using Bilat Blur...")
        #Applying the blur filter
        #img = cv2.GaussianBlur(img,(5,5),0)
        img = cv2.bilateralFilter(img,9,75,75) 
        for ij in range (iblur):
                img = cv2.bilateralFilter(img,9,75,75) 
        
    img =np.array(img)
    img = np.array(img, dtype = np.float16)

    #apply overall threshold - below thresh = 0, creates holes
    img[img <thresh] = 0
    
    if normalize==True:
            minval=np.amin(img[img>0])-clipvalue
            print ("Normalize...", minval, np.amax(img))
            if norm_zero: 
                print ("Start buildup at zero...", minval)
                img=img-minval
                #img[img <0] = 0
                img=np.clip(img, 0, 255)
                #img[img <0] = 0
                #print (img.shape)
                
                #for i in range (img.shape[1]):
                #    for j in range (img.shape[0]):
                #        print (img[i,j])
                #        if img[i,j]<0:
                #            print (img[i,j])
                
                plt.imshow(img, interpolation='nearest',cmap="hot")
                plt.colorbar()
                plt.show()
                #img = [0 if a_ < 0 else a_ for a_ in img]
            print ( np.amin(img), np.amax(img))
        
            img=img/np.amax(img)*255
    
            
    img=img*sat
    img[img >255] = 255
    
    
    img = np.array(img, dtype = np.uint8)

    #number of layers
     
    #here add middle SOLID PLATE
    if darea>0:
        #img=Image.fromarray(img)
        
        #thresh_i = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C  ,cv2.THRESH_BINARY_INV,5,2)
        _, thresh_i = cv2.threshold(img,80,255,cv2.THRESH_BINARY)
        plt.imshow(thresh_i, interpolation='nearest',cmap="hot")
        plt.colorbar()
        plt.show()

        print ("##remove small areas...", darea)
        # Filter using contour area and remove small noise
        cnts = cv2.findContours(thresh_i, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE )
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            #print (area)
            if area < darea:
                #print ("REMOVE")
                cv2.drawContours(img, [c], -1, (0,0,0), -1)
                #cv2.drawContours(img, [c], -1, (255,255,255), -1)
        #img =np.array(img)

    print ("Image to be used...")    
    if invertresult:
        print ("INVERT")
        img = cv2.bitwise_not(img)
    
    plt.imshow(img, interpolation='nearest',cmap="hot")
    plt.colorbar()
    plt.show()
    print ("start for i=", imc)
    
    if invvv==True:
      print ("add inverse on other side") 
      for ii in range (maxheight):
    
        threshold=(maxheight-ii)/(maxheight)*255
        #new_img  = img[img<= threshold]
        z = np.copy(img)
        
        z[z <threshold] = 0
        z[z >0] = 255
            
        vox.append(z)

    if centerrep>0:
        print ("adding center layer/solid!")
        z2 = np.copy(img)

        z2[z2 >=0] = 255
    
        for infg in range( (centerrep)):
             vox.append(z2)
    
    

    for ii in range (maxheight):
    
        threshold=(ii/maxheight)*255
        #new_img  = img[img<= threshold]
        z = np.copy(img)
        
        z[z <threshold] = 0
        z[z >0] = 255
        
            
        vox.append(z)

   return np.array(vox)

def TwoDimg23Dvox_BACKUP(img,maxheight=10, invvv=False, thresh=0, normalize=False,sat=1,GaussSmoothimage=False, BilatSmoothimage=False):

    if GaussSmoothimage==True:
        print ("smoothen using Gaussian Blur...")
        #Applying the blur filter
        img = cv2.GaussianBlur(img,(5,5),0)
        #img = cv2.bilateralFilter(img,9,75,75) 
    if BilatSmoothimage==True:
        print ("smoothen using Gaussian Blur...")
        #Applying the blur filter
        #img = cv2.GaussianBlur(img,(5,5),0)
        img = cv2.bilateralFilter(img,9,75,75) 
        
    img =np.array(img)
    #apply overall threshold - below thresh = 0, creates holes
    img[img <thresh] = 0
    
    if normalize==True:
            
            print ("Normalize...", np.amin(img), np.amax(img))
            img=img-(np.amin(img)   )
            img[img <0] = 0
    
            print (  np.amax(img))
            img=img/np.amax(img)*255
            
    img=img*sat
    img[img >255] = 255
    
    plt.imshow(img, interpolation='nearest',cmap="hot")
    plt.colorbar()
    plt.show()
    print ("start")
    vox = []
    #number of layers
     
    
    if invvv==True:
      print ("add inverse on other side") 
      for ii in range (maxheight):
    
        threshold=(maxheight-ii)/(maxheight)*255
        #new_img  = img[img<= threshold]
        z = np.copy(img)
        
        z[z <threshold] = 0
        z[z >0] = 255
        
        #print (ii,threshold,z )
        
        #   for x in range (img.shape[0]):
        #       for y in range (img.shape[1]):
        #           pixelint=img[y,x]
        #           print (x,y,pixelint)
        
        #plt.imshow(z, interpolation='nearest')
        #plt.show()
            
        vox.append(z)

    for ii in range (maxheight):
    
        threshold=(ii/maxheight)*255
        #new_img  = img[img<= threshold]
        z = np.copy(img)
        
        z[z <threshold] = 0
        z[z >0] = 255
        
        #print (ii,threshold,z )
        
        #   for x in range (img.shape[0]):
        #       for y in range (img.shape[1]):
        #           pixelint=img[y,x]
        #           print (x,y,pixelint)
        
        #plt.imshow(z, interpolation='nearest')
        #plt.show()
            
        vox.append(z)

    return np.array(vox)


def vox2stl(vox, loc='.', filename='', save=True, smooth=False, smooth_iter=20):
    mesh = trimesh.voxel.ops.matrix_to_marching_cubes(vox)

    if smooth:
        print ("Smoothing: on")
        
        #mesh = trimesh.smoothing.filter_humphrey(mesh)
        
        #trimesh.smoothing.filter_mut_dif_laplacian(mesh, lamb=0.5, iterations=10, volume_constraint=True, laplacian_operator=None)
        mesh=  trimesh.smoothing.filter_mut_dif_laplacian(mesh, lamb=0.85, iterations=smooth_iter)
        

    mesh.rezero()
    if save:

        from time import strftime
        stamp = strftime("%m_%d_%H_%M")
        os.makedirs(loc, exist_ok=True)
        mesh.export(loc+'/'+filename+'_'+stamp+'.stl')
        print('save stl model to {}'.format(loc+'/'+filename+'_'+stamp+'.stl'))

# here are four basic logical operations that can be applied to voxels

# return a vox that vox1 OR vox2 exist (A||B) 
def union(vox1, vox2): 
    return np.logical_or(vox1, vox2)

# return a vox that EITHER only vox1 OR vox2 exists (A||B-A&&B)
def xor(vox1, vox2):
    return np.logical_xor(vox1, vox2)

# return a vox that both vox1 exists BUT vox2 does not (A-B) 
def substraction(vox1, vox2):
    return np.logical_xor(np.logical_or(vox1, vox2), vox2)

# return a vox that vox1 and vox2 BOTH exist (A&&B) 
def intersection(vox1, vox2):
    return np.logical_and(vox1, vox2)

# return a vox inverse to the original (B!=A) 
def inverse(vox):
    return np.logical_not(vox)

def repeat(vox, repeatance_array):
    return np.tile(vox, repeatance_array)*1


def vox2img(vox, loc='.', filename=''):
    
    from time import strftime
    stamp = strftime("%m_%d_%H_%M")
    os.makedirs(loc+'/'+filename+'_img/', exist_ok=True)
    for i in range(vox.shape[0]):
        temp_img=vox[i]
        plt.imsave(loc+'/'+filename+'_img/'+filename+'_'+stamp+'_'+str(i)+'.png', temp_img, cmap='gray')
        from IPython import display
        display.clear_output(wait=True)
        plt.imshow(temp_img, cmap='gray')    
        plt.axis('off')
        plt.title(str(i))
        plt.show()
    
    print('save stl model as a stack of images into {}'.format(loc+'/'+filename+'_img/'))

def vox2npy(vox, loc='.', filename=''):
    from time import strftime
    stamp = strftime("%m_%d_%H_%M")
    os.makedirs(loc, exist_ok=True)
    np.save(loc+'/'+filename+'_'+stamp+'.npy', vox)
    print('save stl model as a 3D array to {}'.format(loc+'/'+filename+'_'+stamp+'.npy'))

def npy2vox(filename=''):
    return np.load(filename)*1

In [ ]:
def show_stl2point (loc='./', fname='voxel_0_tester_0_06_11_05_45.stl', 
                    size=64, sizevoxel =1 , reorient=True, 
                    threshold=0.8 , scaled=False, show_on=True ):
    im, STLname=stl2vox(loc,fname, size=size, sizevoxel =sizevoxel , reorient=reorient, threshold=threshold, scaled=scaled  )
    print ("Voxel tensor shape: ", im.shape)
    
    fig = plt.figure(figsize=(6, 6))
    gs = gridspec.GridSpec(1, 1)
    gs.update(wspace=0.05, hspace=0.05)

    x, y, z = im.nonzero() #returns indixes or coordinates of all nonzero entries...
    
    if show_on:
        ax = plt.subplot(gs[0], projection='3d')
        ax.scatter(x, y, z, zdir='z', c='red', marker='s', s=2)
 
        plt.show()

        plt.scatter (x, z, c='red', marker='s', s=2)
        #plt.axis('square')
        plt.axis('equal')
        plt.xlabel('x')
        plt.ylabel('z')    
        plt.show()

        plt.scatter (y, z, c='red', marker='s', s=2)
        #plt.axis('square')
        plt.axis('equal')
        plt.xlabel('y')
        plt.ylabel('z')
        plt.show()

        plt.scatter (x, y, c='red', marker='s', s=2)
        #plt.axis('square')
        plt.axis('equal')
        plt.xlabel('x')
        plt.ylabel('y')

        plt.show()
    
    
    dx=max(x)-min(x)
    dy=max(y)-min(y)
    dz=max(z)-min(z)
    
    print (f"Max/min x {max(x)}, {min (x)}; Max/min y {max(y)}, {min (y)}; Max/min z {max(z)}, {min (z)}")
    print (f"(dx,dy,dz)=({dx}, {dy}, {dz})")
    
    
    return im, [x, y, z], [dx, dy, dz], STLname

In [ ]:

from sklearn.decomposition import PCA
    

def reorient_STL_largestaxis_OLD (fnamein, fnameout):
    stl = mesh.Mesh.from_file(fnamein)


    # method: PCA to find the principal component as the long axis. This may fail for some weird geometries...
    stl.vectors = stl.vectors - np.mean(stl.vectors, axis=0)
    stl.update_centroids()
    pca=PCA(2)
    pca.fit(stl.centroids)
    long_vector=pca.components_[0]


    # In[16]:


    stl.rotate([1, 0, 0], math.acos(long_vector[0]/np.linalg.norm(long_vector)))
    stl.rotate([0, 1, 0], math.acos(long_vector[1]/np.linalg.norm(long_vector)))
    stl.rotate([0, 0, 1], math.acos(long_vector[2]/np.linalg.norm(long_vector)))
    
    #stl.rotate([0, 1, 0], math.radians(90))
    #stl.rotate([0, 1, 0], math.radians(-90))
    stl.rotate([0, 0, 1], math.radians(90))
    
    # stl.vectors = stl.vectors - stl.vectors[0]
    # stl.update_centroids()
    print ("saving rotated STL under: ", fnameout)
    stl.save(fnameout)
    
    return fnameout



def reorient_STL_largestaxis (fnamein, fnameout):
    stl = mesh.Mesh.from_file(fnamein)

     # method: PCA to find the principal component as the long axis. This may fail for some weird geometries...

     
    pca=PCA(2)
    pca.fit(stl.centroids)
    long_vector=pca.components_[0]

    axis=np.cross(long_vector, [0,0,1])
    stl.rotate(axis, -math.acos(np.dot(long_vector, [0,0,1])/np.linalg.norm(long_vector)))
    stl.vectors = np.round(stl.vectors, 4)
    stl.update_centroids()
    #stl.save(fname+'_after.stl')

    print ("saving rotated STL under: ", fnameout)
    stl.save(fnameout)
    
    return fnameout

 


In [ ]:
from trimesh.voxel import creation
#from mesh_to_sdf import mesh_to_voxels
import scipy.ndimage as nd
import trimesh
import skimage
from stl import mesh as stlmesh
from stl import mesh
#!pip install numpy-stl
def find_mins_maxs(obj):
    minx = obj.x.min()
    maxx = obj.x.max()
    miny = obj.y.min()
    maxy = obj.y.max()
    minz = obj.z.min()
    maxz = obj.z.max()
    return minx, maxx, miny, maxy, minz, maxz

def stl2vox(loc='', filename='', size=64, sizevoxel=1., reorient=False, 
            threshold=0.5, scaled=False): #scaled = True = will rescale to unit cube first then transform 
    
    #mesh = trimesh.voxel.ops.matrix_to_marching_cubes(vox)

    if scaled==False:
        if reorient:
            print ("Reorient along longest axis...")
            filenamenew='reor_'+filename
            reorient_STL_largestaxis ( loc+filename, loc+filenamenew)

            filename=filenamenew

        print ("Loading...", loc+filename)
        mesh = trimesh.load_mesh(loc+filename)
        #mesh = trimesh.scale_to_unit_sphere(mesh)
        #voxel_grid=creation.local_voxelize(mesh,mesh.centroid, mesh.extents.max() / (2*size), size, fill=True)
        #tmp_raw=voxel_grid.encoding.data



        #mesh (trimesh.Trimesh) – Source geometry
        #point ((3, ) float) – Point in space to voxelize around
        #pitch (float) – Side length of a single voxel cube
        #radius (int) – Number of voxel cubes to return in each direction.
        #kwargs (parameters to pass to voxelize_subdivide) –
        #print (mesh.centroid)

        voxels=creation.local_voxelize (mesh, mesh.centroid, sizevoxel, size, fill=True).encoding.data*1.
        voxels[voxels <= threshold] = 0

        #voxels = nd.zoom(voxels, ( size/voxels.shape[0],size/voxels.shape[1],size/voxels.shape[2]   ), 
        #                 mode='constant', order=0)

        return voxels, filename

    if scaled:
        if reorient:
            print ("Reorient along longest axis...")
            filenamenew='reor_'+filename
            reorient_STL_largestaxis ( loc+filename, loc+filenamenew)

            filename=filenamenew


        mesh = trimesh.load(loc+filename)
        mesh=scale_to_unit_cube(mesh) 
       
        voxels = creation.local_voxelize (mesh, mesh.centroid, sizevoxel, size, fill=True).encoding.data*1.
        voxels[voxels <= threshold] = 0

        voxels = nd.zoom(voxels, ( size/voxels.shape[0],size/voxels.shape[1],size/voxels.shape[2]   ), 
                         mode='constant', order=0)

        # print ("Voxel size: ",voxels.shape )
        #return creation.voxelize (mesh, pitch=sizevoxel)
        return voxels     , filename
    
def scale_to_unit_cube(mesh):
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump().sum()

    vertices = mesh.vertices - mesh.bounding_box.centroid
    #vertices *= 2 / np.max(mesh.bounding_box.extents)
    vertices *= 1 / np.max(mesh.bounding_box.extents)

    return trimesh.Trimesh(vertices=vertices, faces=mesh.faces)


In [ ]:

import numpy as np

im_res=32

In [ ]:
def load_voxel (path):

    voxels=np.load(path)
    #voxels = nd.zoom(voxels, (.5, .5, .5), mode='constant', order=0)
    
    print (voxels.shape)
    return voxels

In [ ]:
v=load_voxel ('np_vox_0000_0000.npy')
#print (v)
plt.plot (v[32,:,20])
plt.show()

In [ ]:
def im_generate (im_resx, im_resy, x, y, aspect, thick, angle):
    # Create a black image
    img = 255*np.ones((im_resy,im_resx,3), np.uint8)

    cv2.ellipse(img,  (int(x),int(y)),(int(aspect*thick),int(thick)),angle,0,360, (0,0,0) ,-1  )
    #cv.ellipse(img,(256,256),(100,50),0,0,180,255,-1)
    return img
    
    

In [ ]:
#check if pixel in material should form a material 

def is_material (img , i, j, thresh=0):
    
    flag = False 
    
    
    if np.any(img[i, j] >thresh): #if black == no material , can also choose threshold
        #print (img[i, j])
        flag=True
        #print (flag)
    return flag

#check if pixel in material should form a material 

def get_material_type (img , i, j, thresh=0):
    
    mtype= 1
    
    
    if np.any(img[i, j] >thresh): #if black == material type 2, which is soft (white inclusions are STIFF!!)
        #print (img[i, j])
        mtype=2
        #print (flag)
    return mtype



def is_material_voxel (img , i, j, k, thresh=0):
    
    flag = False 
    
    
    if np.any(img[i, j, k] >thresh): #if black == no material , can also choose threshold
        #print (img[i, j])
        flag=True
        #print (flag)
    return flag

#check if pixel in material should form a material 

def get_material_type_voxel (img , i, j, k, thresh=0):
    
    mtype= 1
    
    
    if np.any(img[i, j, k] >thresh): #if black == material type 2, which is soft (white inclusions are STIFF!!)
        #print (img[i, j])
        mtype=2
        #print (flag)
    return mtype

In [ ]:
import random

In [ ]:
def read_result (fname):

    data_list=[]
    read_n=False
    i=0
    countframe=0
    with open(fname) as f:
        for line in f:
                #print (i)
                content_list = line
                if read_n==True:
                        if ("ITEM:" in content_list ):
                            read_n=False
                            countframe=countframe+1
                        else:
                            content_list = line.split(" ")
                            #print ("f", content_list [3])
                            content_list=np.array (content_list,dtype=float) 
                            data_list.append (content_list)
                if (countframe==0) and ("ITEM: ATOMS" in content_list ):
                        read_n=True

    #print (data_list)
    data_list=np.array (data_list,dtype=float) 

    
    return (data_list)

def save_im (data_list, name , jj, color=True, coll=11, size=25,vmin=0., vmax=1.,marker='h', data_source=None, showw=False):
    #fname='./LAMMPS_OUT_FILES/dump_small_mobile.stress'
    #data_list =read_result (fname)
    size_=size
    #print (len(data_list) )
    #plt.scatter(x, y, c=z, s=5, cmap=colormap, norm=normalize, marker='*')

    stdev_=0
    variance_=0
    mean_=0
    plt.figure(figsize=(10,10))
    #print (data_list.shape )
    if color== True:    
        colormap = plt.cm.jet #or any other colormap
        normalize = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
        if coll>4 and coll<20:
            
            data_list_data=data_list
            if not data_source is None:
                data_list_data=data_source
            plt.scatter(data_list[:, 1], data_list[:, 2], c=data_list_data[:, coll], \
                        marker=marker, s=size_, cmap=colormap, norm=normalize );
            stdev_=statistics.stdev (data_list_data[:, coll])
            variance_=statistics.variance (data_list_data[:, coll])
            mean_=statistics.mean (data_list_data[:, coll])
            #plt.tricontourf(data_list[:, 1], data_list[:, 2],  data_list[:, coll], cmap='jet');
        if coll==4:
            plt.scatter(data_list[:, 1], data_list[:, 2], c=data_list[:, coll], marker=marker, s=size_, cmap='CMRmap');
        
             
        if coll==20: #x displacement
            
            plt.scatter(data_list[:, 1], data_list[:, 2], c=data_list[:, 1]-data_source[:, 1], marker=marker, s=size_, cmap=colormap, norm=normalize );
        if coll==21: #y displacement
            
            plt.scatter(data_list[:, 1], data_list[:, 2], c=data_list[:, 2]-data_source[:, 2], marker=marker, s=size_, cmap=colormap, norm=normalize );
        if coll==22: #y displacement
            
            plt.scatter(data_list[:, 1], data_list[:, 2], c=data_list[:, 3]-data_source[:, 3], marker=marker, s=size_, cmap=colormap, norm=normalize );
            
            #plt.tricontourf(data_list[:, 1], data_list[:, 2],  data_list[:, coll]);
    if color== False:    
        plt.scatter(data_list[:, 1], data_list[:, 2], c="black" , marker=marker, s=size_, cmap='jet');
    
    plt.subplots_adjust(left=0., bottom=0., right=1., top=1.)
    plt.axis('square')
    plt.axis('off')
    
    #fig = plt.figure()
    #plt.figure().subplots_adjust(
    #top=0.0,
    #bottom=0.0,
    #left=0.0,
    #right=1.0,
    #hspace=0.0,
    #wspace=0.0
    #)
    
    #fig.tight_layout()
    #jj=10
    #plt.set_size_inches(5, 8)
    
    
    final_name= name+"%5.5d.png"%jj
    
    #plt.savefig(name+"%5.5d.png"%jj, dpi=100,pad_inches=0, bbox_inches='tight' ) #save as png
    temp="temp.png"
    plt.savefig(temp, dpi=200,pad_inches=0  ) #save as png
    
    if showw==True:
        plt.show()
    #cv2.imwrite('/path/to/destination/image.png',image)
    
    
     
    print ("Open image....") 
    # Opens a image in RGB mode
    im = Image.open(temp,mode='r')
    width, height = im.size
    cropp=width*.04
    # Setting the points for cropped image
    left = cropp
    top = cropp
    right = width -cropp 
    bottom = height -cropp

    # Cropped image of above dimension
    # (It will not change original image)
    im1 = im.crop((left, top, right, bottom))
    
    im1 = im1.resize( (width, height) )
    
    im1.save(final_name,  format="png")

    return final_name, stdev_, variance_, mean_

def same_orig_im (forpx,forpy,name, jj ):
    
 
    plt.scatter(forpx , forpy  ,marker='s', s=1, c="black" )
    #plt.axis('off')
                
    final_name= name+"%5.5d.png"%jj
    plt.savefig(final_name, dpi=200,pad_inches=0 ) #save as png
    plt.axis('square')
    plt.show()
    return final_name    
                

In [ ]:
from collections import OrderedDict
        

#takes lists earlier generated and writes LAMMPS coor file
def gen_LAMMPS_cor_voxel (forpx, forpy,forpz, atomlist, bondlist, pbcx,pbcy,pbcz,
                          outname='./data_harmonic.coor'):
           
        

    
    bnumber = len(bondlist)-1
    header=[]

    header.append(('\n%d atoms \n'% (len(atomlist)-1) ))
    header.append(('%d bonds \n'% bnumber))
    #header.append ('%d angles \n'% anglenumber )

    header.append ('\n' )

    header.append (('%d atom types \n'% 1 ))
    header.append (('%d bond types \n'% 1 ))
    #header.append (('%d angle types \n'% 1 ) )


    header.append ('\n' )

    header.append (f'{-pbcx} {pbcx}    xlo xhi \n' )
    header.append (f'{-pbcy} {pbcy}    ylo yhi \n' )
    header.append (f'{-pbcz} {pbcz}    zlo zhi \n' )

    header.append ('\n\n' )

    header.append ('Masses \n\n')
    header.append ('1  1.00 \n')
   
    header.append ('\n' )

#1 12.01
#2 1.00794

    #####################
    # write file


    file = open(outname,"w")

    file.writelines(header)
    file.writelines(atomlist)
    file.writelines(bondlist)
    #file.writelines(anglelist)


    file.close()
    return

#this must generate atom lists, bond lists etc. to be written later
def gen_LAMMPS_coord_from_voxel (im, r0_lattice_x, r0_lattice_y,r0_lattice_z, 
                                 ux, uy, uz, attype, precise=True, 
                                 r_cut=1.1, delete_unbonded=False):

    #before: image = number of unit cells
    #maxx=im.shape[1]
    #maxy=im.shape[0]
    
    
    if precise:
        maxx=int (im.shape[0]/r0_lattice_x)
        maxy=int (im.shape[1]/r0_lattice_y)
        maxz=int (im.shape[2]/r0_lattice_z)
    else:
        maxx=  im.shape[0] 
        maxy=  im.shape[1] 
        maxz=  im.shape[2] 
    
    #spacing of lattice

    #UNIT: Angstroms

    sizexx=maxx*1
    sizeyy=maxy*1
    sizeyz=maxz*1
    
    print ("-------------------------------------------------------------------------")
    print ("Image dimensions: ", maxy, maxx, maxz)
    print ("-------------------------------------------------------------------------")
    atomlist=[]
    bondlist =[]
    anglelist =[]


    forp=[]
    forpx=[]
    forpy=[]
    forpz=[]
    forptype=[] 

    ucellsize=len (ux)
    anumber = 0    
    bnumber = 0
    anglenumber =0   
    for i in range ( int(maxx)):
        if i%100==0:
            print ("Now checking i= ", i)
        for j in range (int(maxy)):
            for k in range (int(maxz)):
                
                for kk in range ( ucellsize):

                    #print (kk)  
                    dx=ux[kk]
                    dy=uy[kk]
                    dz=uz[kk]

                    #atype=1
                    #molid=1

                    #print (i, j)
                    #print (kk, dx, dy)
                    xxx=i*r0_lattice_x +dx
                    yyy=j*r0_lattice_y +dy
                    zzz=k*r0_lattice_z +dz

                    #add=False
                    if precise:
                        atype=get_material_type_voxel  (im , int (xxx), int (yyy), int (zzz)  )
                    else:
                        atype=get_material_type_voxel  (im , int (i), int (j), int (k)  )
                    
                    #atype=get_material_type_voxel  (im , int (xxx), int (yyy), int (zzz)  )
                    #add=is_material (im , j, i) 
                    #print (atype)

                    if atype>1:     
                        #print ("***", add)
                        anumber=anumber+1
                        attype_=attype[kk] # get particular atom type from unit cell - need to change if use multispecies LJ...
            #            chh= ("%d %d %d %f %f %f \n") % (anumber, molid, atype, 
            #                                             xxx,yyy,zzz)
                        molid=1
                        chh= ("%d %d %d %f %f %f \n") % (anumber, molid, attype_, xxx,yyy,zzz)
            
                        #chh= ("%d  %d  %10.4f  %10.4f  %10.4f\n") % (anumber,   attype_, 
                        #                                 xxx,yyy,zzz)
                        atomlist.append ( chh)

                        forp.append( [xxx,yyy, zzz])
                        forpx.append( xxx)
                        forpy.append( yyy)
                        forpz.append( zzz)
                        
                        forptype.append(atype)
                #if add==False:    # now we are in INTERIOR of channel... the fluid  

                
                
    print ("Number atoms added: ", len(forpx) )        
    
    
    #now generate bonds
    
    #first, translate list to np array for fast comp
    forp_np = np.array(forp)

    print ("Shape coordinate list: ", forp_np.shape)
    
    ######################### Generate bond list....
    #euclidean_distance = np.linalg.norm(np.array(features)-np.array(predict))
    #        print(euclidean_distance)
    bonded=[]
        
    bnumber = 0
    for i in range ( forp_np.shape[0]):
        
        x0=forp_np[i, 0]
        y0=forp_np[i, 1]
        z0=forp_np[i, 2]
        
        if i%1000==0:
            print ("Progress: ", i)
        
        j=i+1
        while j<  forp_np.shape[0]:
    
            x1=forp_np[j, 0]
            y1=forp_np[j, 1]
            z1=forp_np[j, 2]
            #r=10
            r= np.sqrt((x1-x0)**2 + (y1-y0)**2 + (z1-z0)**2)
            
            #print (r)
            
            if r < r_cut and j>i: #only add bonds ONCE
                #print ("bond found: ", i+1, j+1, r)
                
                btype=1
                bnumber=bnumber+1 

                chh= ("%d  %d    %d %d  \n") % (bnumber, btype,  i+1, j+1 )
                bondlist.append ( chh)  
                
                bonded.append(i)
                bonded.append(j)
                
            j=j+1
            
    if delete_unbonded:        
        #remove atoms not bonded
        dellist=[]
        for i in range ( forp_np.shape[0]):
            if i not in bonded:

                dellist.append(i)



        dellist=list(OrderedDict((element, None) for element in dellist))

        print ("to be deleted.... ", dellist)

        for ele in sorted(dellist, reverse = True):
            
            '''
            del atomlist[ele]
            del forp[ele]
            del forpx[ele]
            del forpy[ele]
            del forpz[ele]
            del forptype[ele]
            '''
            print("delete unbonded particles doesnt work yet... need to renumber particles")
            
            #chh[ele]= ("%d %d %d %f %f %f \n") % (ele, molid, 2, xxx,yyy,zzz)
             
            
    atomlist.insert (0, 'Atoms \n\n')
          
    bondlist.insert (0,'\nBonds \n\n')
    
    
    return atomlist, bondlist, forp, forpx, forpy, forpz, forptype



In [ ]:
dest = './Voxel_runs/'
os.makedirs(dest , exist_ok=True)

In [ ]:
!set OMP_NUM_THREADS=4

In [ ]:
v=load_voxel ('np_vox_0000_0000.npy')

im_resx = v.shape[0]
im_resy = v.shape[0]
im_resz = v.shape[0]


 

In [ ]:
def replace_string (data,rp1, rp1_n):
    data=data.replace(rp1, rp1_n)
    return data
 
def set_sim_parameters (fname="in_python_FCC_harmonic_reference_pull.txt", 
                        fnameout="in_python_FCC_voxel_harmonic_pull_PROCESSED.txt",
                       k_spring=10., r0=5.):
    text_file = open(fname, "r")
    #read whole file to a string
    data = text_file.read()
    #close file
    text_file.close()
    #print(data)
    
    print ("Set FF parameters...", k_spring, r0)

    #DONT NEED FOR PBCs... just leave here anyway
    

    b_1='bond_coeff	1 100 1.'
    b_1_n=f"bond_coeff    1 {k_spring:.5f} {r0:.5f}"
    
    
    data=replace_string (data,b_1, b_1_n)
    

    
    
    # LJ potential 
    b_1='#pair_style	lj/cut 0.8'
    b_1_n=f"pair_style	lj/cut {r0:.5f}" #cutoff at r_0 so only have repulsive part
    data=replace_string (data,b_1, b_1_n)
    
    b_1='#pair_coeff	* * 0. 1.'
    sigma = r0/ (2**(1/6.))
    eps = k_spring/5
    b_1_n=f"pair_coeff * * {eps} {sigma}"  #sigma r0= 2**(1/6.) * sigma  --- sigma = r0/ (2**(1/6.))
    data=replace_string (data,b_1, b_1_n)
    
    
    
    
    
    #k ... cutoff
    #b_1='pair_coeff * * 0.2 2.0'
    #b_1_n=f"pair_coeff * * {k_spring:.5f} {r0:.5f}"
    

    #data=replace_string (data,b_1, b_1_n)
    

    #print (data)
    text_file = open(fnameout, "w")
    #read whole file to a string
    text_file.write(data)
    #close file
    text_file.close()
    

def setup_FCC_harmonic (r0=4.5 ):





    #r0_lattice=.93*2

    #r0_lattice = 2**0.5 * r0
    r0_lattice = 2**0.5 * r0

    #print (r0, r0_lattice)
    #harmonic
    r0_lattice_x=r0_lattice
    r0_lattice_y=r0_lattice
    r0_lattice_z=r0_lattice



    ux = [0,             0  ,                  0.5*r0_lattice,           0.5*r0_lattice]
    uy = [0.,            0.5*r0_lattice,       0.  ,                    0.5*r0_lattice ] 
    uz = [0,             0.5*r0_lattice,       0.5*r0_lattice,            0] 

    #coordinates
    #0,0,0; 0,1/2,1/2; 1/2,0,1/2; and 1/2,1/2,0


    r0_lattice_x=r0_lattice
    r0_lattice_y=r0_lattice
    r0_lattice_z=r0_lattice
    ucellsize = len (ux)

    attype = [1,1,1,1 ]
    #print ("Unit cell dimension in x,y: ", r0_lattice_x, r0_lattice_y, r0_lattice_z)

    return ux, uy, uz, r0_lattice_x, r0_lattice_y, r0_lattice_z, ucellsize, attype


    #LJ
    #hexagonal
    #r0_lattice_x=r0_lattice*2
    #r0_lattice_y=r0_lattice * sqrt(3. )
    #ucellsize=4
    #ux = [0, r0_lattice/2,r0_lattice, r0_lattice*3/2]
    #uy = [0, r0_lattice*sqrt(3)/2.,0, r0_lattice*sqrt(3)/2.   ]

    #graphene
    '''
    sqrt3= sqrt(3)
    bondl=1.42
    r0_lattice_x=sqrt3*bondl * 2 ## two copies in x
    r0_lattice_y=3*bondl

    ucellsize=8
    ux = [0,  0.5*sqrt3*bondl, 0.5*sqrt3*bondl, 0.0, \
          0+r0_lattice_x/2.,  0.5*sqrt3*bondl+r0_lattice_x/2., 0.5*sqrt3*bondl+r0_lattice_x/2., 0.0+r0_lattice_x/2.]
    uy = [0.5*bondl, bondl ,2*bondl, 2.5*bondl,.5*bondl, bondl ,2*bondl, 2.5*bondl   ]

    attype = [1,1,1,1, 1,1,1,1]
    print ("Unit cell dimension in x,y: ", r0_lattice_x, r0_lattice_y)
    '''

In [ ]:


def set_pulling (filename="in_python_FCC_voxel_harmonic_pull_PROCESSED.txt", low_z=20, high_z=60,  deltaz=10,
                 vx=0.0, vy=0.0, vz=0.1, pullsteps=20000, timestep = 0.075, LAMMPS_OUT = './LAMMPS_OUT_FILES/'):

    text_file = open(filename, "r")
    #read whole file to a string
    data = text_file.read()
    #close file
    text_file.close()
    #print(data)

    #region....xlo xhi ylo yhi zlo zhi
    #print ("Data provided: ", pullsteps, vx,vz,vz, low_z, high_z, deltaz)
    
    #vx=0.0
    #vy=0.0
    #vz=0.1
    
    #ramp command
    
    #value = x + (y-x) * (timestep-startstep) / (stopstep-startstep)
    
    data=replace_string(data,rp1= "timestep 0.075",  
                   rp1_n=f"timestep     {timestep}")
   
    #DONT NEED FOR PBCs... just leave here anyway
    data=replace_string(data,rp1= "#region	        1 block INF INF INF 10 INF INF",  
                   rp1_n=f"region	        1 block INF INF INF INF INF  {low_z+deltaz} ")
    
    data=replace_string(data,rp1= "#group		lower region 1",  
                   rp1_n=f"group		lower region 1")
    
    data=replace_string(data,rp1="#region		2 block INF INF 118 INF INF INF",
                   rp1_n=f"region	        2 block INF INF INF INF {high_z-deltaz} INF")
                        
    data=replace_string(data,rp1= "#group		upper region 2",  
                   rp1_n=f"group		upper region 2")
    
    data=replace_string(data,rp1= "#group		boundary union lower upper",  
                   rp1_n=f"group		boundary union lower upper")
    
    data=replace_string(data,rp1= "#group		mobile subtract all boundary",  
                   rp1_n=f"group		mobile subtract all boundary")
    
    
    
    
    data=replace_string(data,rp1= "#velocity	upper set 0.0 0.3 0.0",  
                   rp1_n=f"velocity	upper set {vx} {vy} {vz}")
   
    data=replace_string(data,rp1= "#velocity	lower set 0.0 0.3 0.0",  
                   rp1_n=f"velocity	lower set {-vx} {-vy} {-vz}")
    
    maxdisplacement = pullsteps*deltaz*timestep # total displacements after pulling steps...
    
    data=replace_string(data,rp1= "#run		20000",  
rp1_n=f'variable 	 disp equal \"ramp(0, {maxdisplacement})\" \n\
variable 	 ssxx equal \"-pxx\" \n\
variable 	 ssyy equal \"-pyy\" \n\
variable 	 sszz equal \"-pzz\"\n\
variable 	 ssxy equal \"-pxy\" \n\
variable 	 ssxz equal \"-pxz\" \n\
variable 	 ssyz equal \"-pyz\" \n\
\n\n\
fix          def1 mobile print 1000 \"${{disp}} ${{ssxx}} ${{ssyy}} ${{sszz}} ${{ssxy}} ${{ssxz}} ${{ssyz}} \" file {LAMMPS_OUT}/sscurve.dat screen no \n\
\n\n\
run		{pullsteps}  \n\n')

#run		{pullsteps} start 0 stop {pullsteps}\n\n')

    data=replace_string(data,rp1= "#fix		2 boundary setforce 0. 0. 0.",  
                   rp1_n=f"fix		200 boundary setforce 0. 0. 0.")
    #data=replace_string(data,rp1= "#fix		2 boundary setforce 0. 0. 0.",  
    #               rp1_n=f"fix		200 boundary setforce NULL NULL 0.")
   
    
    
   
    print ("####################################################################")
    print ("## PULLING ADDED...")
    #print (data)
    print ("####################################################################")
    text_file = open(filename, "w")
    #read whole file to a string
    text_file.write(data)
    #close file
    text_file.close()

In [ ]:
 
def append_enmin ( fname="in_python_FCC_voxel_harmonic_pull_PROCESSED.txt", steps=1000, 
                  maxeval=10000, LAMMPS_OUT='./LAMMPS_OUT_FILES/'):
    text_file = open(fname, "r")
    #read whole file to a string
    data = text_file.read()
    #close file
    text_file.close()
    #print(data)
    
    vx=0
    vy=0
    vz=0
    
    data=replace_string(data,rp1= "stop_simulation",  
                   rp1_n=f"velocity	lower set {-vx} {-vy} {-vz}\nvelocity	upper set {-vx} {-vy} {-vz}\n \nstop_simulation")
    
    
    
    data=replace_string(data,rp1= "stop_simulation",  
                   rp1_n=f"min_style cg\nminimize        1e-8 1e-8 {steps} {maxeval}\n\nstop_simulation")
    
    
    
    str_=f'###################FINAL STRESS FIELD#####################################\n\
dump stressfinal all custom 1 ./{LAMMPS_OUT}/dump_final.stress id x y z v_sxx v_syy v_szz v_sxy v_sxz v_syz v_mises\n\
####################################################################\n\nrun 0\n\nstop_simulation'

    data=replace_string(data,rp1= "stop_simulation",  
                   rp1_n=str_)

    str_='###################FINAL STRESS FIELD ONLY MOBILE#####################################\n\
dump stressfinal_mobile mobile custom 1 ./LAMMPS_OUT_FILES/dump_final_mobile.stress id x y z v_sxx v_syy v_szz v_sxy v_sxz v_syz v_mises\n\
####################################################################\n\nrun 0\n\nstop_simulation'

    data=replace_string(data,rp1= "stop_simulation",  
                   rp1_n=str_)

    

    #data.append (f"min_style cg\n minimize        1e-8 1e-8 {steps} {maxeval}\n\nstop")    
    #print (data)
    text_file = open(fname, "w")
    #read whole file to a string
    text_file.write(data)
    #close file
    text_file.close()

In [ ]:
def fix_output_directory ( fname="in_python_FCC_voxel_harmonic_pull_PROCESSED.txt", 
                          ORIGINAL='./LAMMPS_OUT_FILES/', LAMMPS_OUT = './LAMMPS_OUT_FILES_NEW/', coordname='./data_harmonic_hres.coor'):
    text_file = open(fname, "r")
    #read whole file to a string
    data = text_file.read()
    #close file
    text_file.close()
    #print(data)
    
    vx=0
    vy=0
    vz=0
    
    data=replace_string(data,rp1= ORIGINAL,  
                   rp1_n=LAMMPS_OUT)
    
    data=replace_string(data,rp1= "./data_harmonic.coor",  
                   rp1_n=coordname)
    
    
    
    
    
    #data.append (f"min_style cg\n minimize        1e-8 1e-8 {steps} {maxeval}\n\nstop")    
    #print (data)
    text_file = open(fname, "w")
    #read whole file to a string
    text_file.write(data)
    #close file
    text_file.close()

In [ ]:
def analyze_plot_stress (fname='./LAMMPS_OUT_FILES/sscurve.dat', miniter= 0, 
                         maxiter=100 , modulus_dir=3, i=0, show_on=True, LAMMPS_OUT='./' ):
    stressdata=pd.read_csv(fname, delimiter='\s+',skiprows=1)
    print ("Stress data shape: ", stressdata.shape)
    #print (stressdata)
    #${disp} ${ssxx} ${ssyy} ${sszz} ${ssxy} ${ssxz} ${ssyz}
    if show_on:

        plt.plot (stressdata.iloc[:,0], stressdata.iloc[:,1], label='$\sigma_{xx}$')
        plt.plot (stressdata.iloc[:,0], stressdata.iloc[:,2], label='$\sigma_{yy}$')
        plt.plot (stressdata.iloc[:,0], stressdata.iloc[:,3], label='$\sigma_{zz}$')
        plt.plot (stressdata.iloc[:,0], stressdata.iloc[:,4], label='$\sigma_{xy}$')
        plt.plot (stressdata.iloc[:,0], stressdata.iloc[:,5], label='$\sigma_{xz}$')
        plt.plot (stressdata.iloc[:,0], stressdata.iloc[:,6], label='$\sigma_{yz}$')

        plt.legend()

        outname =   f'{LAMMPS_OUT}/stressplot_{i}.jpg'#f"./HIST_epoch_high-{epoch+start_h}-option-{epoch}.jpg"
        plt.savefig(outname, dpi=300)
        plt.show()

    #calculate modulus
    modulus= (stressdata.iloc[maxiter,modulus_dir]-stressdata.iloc[miniter,modulus_dir])/(maxiter-miniter)
    print (f"Modulus in {modulus_dir} direction = {modulus} - tangent at {miniter}..{maxiter}")
    
    max_stress= max(stressdata.iloc[:,modulus_dir])
    print (f"Max stress in {modulus_dir} direction = {max_stress}")
           
    return modulus, max_stress
           


In [ ]:
def analyze_stress_field(fname='./LAMMPS_OUT_FILES/dump_final.stress' , i=0, 
                         ilocal=10, show_on=True, LAMMPS_OUT='./'  ):
    #  id x y z v_sxx v_syy v_szz v_sxy v_sxz v_syz v_mises
    #  0  1 2  3  4     5     6     7     8     9     10
    stressfield=pd.read_csv(fname, delimiter='\s+',skiprows=9)#skip first 9 rows... header...
    
    if show_on:

        #print (stressfield)
        fig_handle = sns.distplot(stressfield.iloc[:,ilocal],bins=30,kde=False, 
                                  rug=False,norm_hist=False,axlabel='Stress' )
        fig = fig_handle.get_figure()
        fig.savefig(f'{LAMMPS_OUT}/stress_dist_iteration-{i}_stresscomponent-{ilocal}.png',dpi=300)
        plt.show()
    
    s_mean = np.mean (stressfield.iloc[:,ilocal])
    s_std = np.std (stressfield.iloc[:,ilocal])
    s_var = np.var (stressfield.iloc[:,ilocal])
    
    print (f"Stress mean={s_mean}, stress stanard dev={s_std}, stress variance={s_var}")
    
    return s_mean, s_std, s_var, stressfield
 

In [ ]:
#s_mean, s_std, s_var, stressfield=analyze_stress_field(fname='./LAMMPS_OUT_FILES/dump_final.stress' , i=0 , ilocal=10) #ilocal = entry of stress tensor...

In [ ]:
def write_data (  modulus_list , max_stress_list , smean_list ,  
                             stddev_list , svar_list , voxellist, pointcloudlist,  STLfilename, loc,
               fname="output_results.csv"):
    
    print ("Write CSV....")
    file = open(fname, "w+", newline='')
    writer = csv.writer(file)
    header = ['modulus', 'max_stress','stress_mean', 's_stddev', 's_var', 'voxel_file', 'pointcloud_file', 
              'STL_file', 'STL_file_loc']
    writer.writerow(header)
    print ("Write data...")
    for w in range(len (modulus_list)): #iterate through  

          writer.writerow  ([modulus_list [w], max_stress_list [w], smean_list [w],  
                             stddev_list [w], svar_list [w], voxellist[w], pointcloudlist [w],  STLfilename[w], loc ])

    file.close()
    
    #return 

In [ ]:

import time
 
    
def run_sims (size_voxel_set=64,#note = 64 in all directions, so actual size is double
             threshold=0.8,
             sizevoxel =1.,
             k_spring= 5.,
             r0=3.5,
             use_omp = False ,
              loc='./STL_collection/',
              STL_fnames=None,
              startjj=0,
             smean_list=[],
            svar_list=[],
            stddev_list=[],
            max_stress_list=[],
            modulus_list=[],
            voxellist=[],
            pointcloudlist=[],
              STLfilename=[],
               csv_fname="output_results_averaged.csv",
             LAMMPS_OUT='./LAMMPS_OUT_FILES/',
              pullsteps=500000,
              vspeed=(0, 0, 0.005),
              coord_name='./data_harmonic_highres.coor',
             show_on=False ,
             reorient=False,
             scaled=False,
             fnameout="in_python_FCC_voxel_harmonic_pull_PROCESSED_highres.txt"):

    os.makedirs(LAMMPS_OUT, exist_ok=True)

    if STL_fnames==None:
        print ("ERROR - no STL files provided....")
        return 
    
    #pbcx=r0_lattice_x*im_resx#*r0_lattice_x
    #pbcy=r0_lattice_y*im_resy#*r0_lattice_y
    #pbcz=r0_lattice_z*im_resz
    pbcx=size_voxel_set *2#*r0_lattice_x
    pbcy=size_voxel_set *2#*r0_lattice_y
    pbcz=size_voxel_set *2





    #safefac=1.35
    #threshold =-0.5
    #threshold =0.05

    #print ("min/max in image dims", xmin, xmax, ymin, ymax )

    #name_list_CSV=[]

    #STL_fnames = ['gyroid_test_2.obj']
    #STL_fnames = ['voxel_0_tester_0_06_11_05_45.stl']
    #STL_fnames = ['voxel_0_tester_0_06_11_05_45.stl','voxel_0_tester_0_06_11_05_45.stl']

    #STL_fnames = ['voxel_0_tester_0_06_11_05_45.stl']

    #print (STL_fnames)

    num_sims = len (STL_fnames)

    
    
    for jj in range (0, num_sims):
        print ("CASE SIMULATION: ", jj , " out of: ", num_sims)
        
        

        ux, uy, uz, r0_lattice_x, r0_lattice_y, r0_lattice_z, ucellsize, attype= setup_FCC_harmonic (r0=r0 )

      
        print ("Considering: ", STL_fnames[jj])

        im, _, _, STLfilename_=show_stl2point (loc=loc, fname=STL_fnames[jj], 
                                 size=size_voxel_set, sizevoxel =sizevoxel , 
                                 reorient=reorient, threshold=threshold, scaled=scaled, show_on=show_on  )

        STLfilename.append(STLfilename_)
     

        atomlist, bondlist, forp, forpx, forpy, forpz, forptype=gen_LAMMPS_coord_from_voxel (im,
                                                                                  r0_lattice_z, 
                                                                                              delete_unbonded=True)

        if show_on:
            print ("###################################################################################")
            print ("PLOT ACTUAL ATOM GEOMETRY....")
            print ("###################################################################################")
            print("Number of atoms generated: ", len(atomlist))
            #print (anumber)
            print ("max x: ", max (forpx), "max y: ", max(forpy), "max z: ", max(forpz))
            print ("min x: ", min (forpx), "min y: ", min(forpy), "min z: ", min(forpz))

            print ('extensions x, y, z', max (forpx)-min (forpx),  max (forpy)-min (forpy), max (forpz)-min (forpz)) 

        if show_on:
            plt.scatter (forpx, forpz, c='red', marker='s', s=2)
            plt.axis('square')
            plt.show()

            plt.scatter (forpy, forpz, c='red', marker='s', s=2)
            plt.axis('square')
            plt.show()

       
        gen_LAMMPS_cor_voxel (forpx, forpy, forpz, atomlist, bondlist, pbcx, pbcy, pbcz, 
                              outname=coord_name)

        
        if show_on:

            gs = gridspec.GridSpec(1, 1)
            gs.update(wspace=0.05, hspace=0.05)

            ax = plt.subplot(gs[0], projection='3d')
            ax.scatter(forpx, forpy, forpz, zdir='z', c='red', marker='s', s=2)
            ax.set_xticklabels([])
            ax.set_yticklabels([])

            ax.set_xlim((0,pbcx))
            ax.set_ylim((0,pbcy))
            ax.set_zlim((0,pbcz))


            plt.show ()

        set_sim_parameters (fname="in_python_FCC_harmonic_reference_pull.txt", 
                            fnameout=fnameout,
                           k_spring=k_spring, r0=r0)
        timestep = 0.075
        #set_pulling (filename="in_python_FCC_voxel_harmonic_pull_PROCESSED.txt", 
        #             low_z= min(forpz), high_z= max(forpz),  deltaz=5.,
        #             vx=0.0, vy=0.0, vz=0.001, pullsteps=300000,  timestep = timestep) 
        
        if show_on:
            print (min(forpz),   max(forpz),  5.)
            #vspeed=(vx=0, vy=0, vz=0.005),
        set_pulling (filename=fnameout, 
                     low_z= min(forpz), high_z= max(forpz),  deltaz=5.,
                     vx=vspeed[0], vy=vspeed[1], vz=vspeed[2], pullsteps=pullsteps,  timestep = timestep,
                     LAMMPS_OUT = LAMMPS_OUT) 

        append_enmin ( fname=fnameout, steps=1000, maxeval=10000, LAMMPS_OUT=LAMMPS_OUT)
        ###############
        #RUN LAMMPS
        #_3 is with higher strain
        
        
        fix_output_directory ( fname=fnameout, 
                          ORIGINAL='./LAMMPS_OUT_FILES/', LAMMPS_OUT = LAMMPS_OUT,
                             coordname=coord_name)
        
        print ("RUN LAMMPS...")
        
        try:

    

            start = time.time()

            if use_omp==False:
            #!lmp.exe -sf omp -pk omp 4 -in ./in_python_FCC_voxel_harmonic_PROCESSED.txt
                print ('NO OMP')
                !lmp.exe  -screen none -in $fnameout 

            if use_omp:
                print ('OMP')
                !lmp.exe  -screen none -sf omp -pk omp 4 -in $fnameout 
                
                
                
            end = time.time()
            print ("###################################################################################")
            print (f"LAMMPS took {(end-start)/60.} minutes....")
            print ("###################################################################################")

            ############ ANALYZE DATA
            print ("stress analysis....")
            modulus, max_stress=analyze_plot_stress (fname=f'{LAMMPS_OUT}/sscurve.dat', 
                                                     miniter= 0, maxiter=10 , modulus_dir=3, i=jj+startjj ,
                                                    show_on=show_on, 
                                                     LAMMPS_OUT=LAMMPS_OUT)
    
            print ("stress stat analysis....")
            s_mean, s_std, s_var, stressfield=analyze_stress_field(fname=f'{LAMMPS_OUT}/dump_final_mobile.stress' , 
                                                                   i=jj+startjj , ilocal=10,
                                                                   show_on=show_on,
                                                                  LAMMPS_OUT=LAMMPS_OUT) #ilocal = entry of stress tensor...

            if show_on:
                print ("Copy....")

           
            
            
            #print ("-")
            shutil.copy(f'{LAMMPS_OUT}/dump_final.stress', f'{LAMMPS_OUT}/dump_final_{jj+startjj}.stress')
            os.remove(f'{LAMMPS_OUT}/dump_final.stress')
            

            #print ("-")
            shutil.copy(f'{LAMMPS_OUT}/dump_final_mobile.stress', f'{LAMMPS_OUT}/dump_final_mobile_{jj+startjj}.stress')
            os.remove(f'{LAMMPS_OUT}/dump_final_mobile.stress')

            #print ("-")
            shutil.copy(f'{LAMMPS_OUT}/sscurve.dat', f'{LAMMPS_OUT}/sscurve_{jj+startjj}.dat')
            os.remove (f'{LAMMPS_OUT}/sscurve.dat')

            #print ("-")
            shutil.copy(f'{LAMMPS_OUT}/dump_small_mobile.stress', f'{LAMMPS_OUT}/dump_small_mobile_{jj+startjj}.stress')
            os.remove(f'{LAMMPS_OUT}/dump_small_mobile.stress')

            #print ("-")
            shutil.copy(f'{LAMMPS_OUT}/dump_small_all.stress', f'{LAMMPS_OUT}/dump_small_all_{jj+startjj}.stress')
            os.remove(f'{LAMMPS_OUT}/dump_small_all.stress')

            #print ("-")
            shutil.copy(f'{LAMMPS_OUT}/dump_initial.geom', f'{LAMMPS_OUT}/dump_initial_{jj+startjj}.geom')
            os.remove(f'{LAMMPS_OUT}/dump_initial.geom')            
            
            
            

            ##################### WRITE VOXEL AND POINTCLOUD DATA 
            f=f'{LAMMPS_OUT}/'+f'np_vox_{jj+startjj}'
            #print ("save ", f)
            #f=outfolder+f'{exc_name}'
            np.save(f, im)
            voxellist.append(f+'.npy')
            if show_on:
                print ("Done copy...")


            f=f'{LAMMPS_OUT}/'+f'np_pointcloud_{jj+startjj}'
            if show_on:
                print ("save ", f)
            #f=outfolder+f'{exc_name}'
            np.save(f, np.array(forp))
            pointcloudlist.append(f+'.npy')

            smean_list.append(s_mean)
            stddev_list.append(s_std)
            svar_list.append(s_var)
            modulus_list.append(modulus)
            
            if show_on:
                print ("max stress: ", max_stress)
            max_stress_list.append (max_stress)


            if show_on:
                print ("CSV file...")

            write_data (modulus_list= modulus_list, max_stress_list =max_stress_list, smean_list =smean_list,  
                             stddev_list =stddev_list, svar_list = svar_list , voxellist = voxellist, 
                        pointcloudlist =pointcloudlist,  STLfilename=STLfilename,loc=loc,
                       fname=csv_fname ) # write every step just to make sure....
            if show_on:
                print ("Done...")

        except:
            print ("Error occurred for jj=", jj)

    return smean_list,svar_list,stddev_list,max_stress_list,modulus_list,voxellist,pointcloudlist,STLfilename


In [ ]:

im, _, _,_=show_stl2point (loc='./STL_objects/', fname='gyroid_test.obj', size=64, sizevoxel =.01 , 
                         reorient=False, threshold=0.8,scaled=True,  )

_,_,_,_=show_stl2point (loc='./', fname='voxel_0_tester_0_06_11_05_45.stl', size=64, sizevoxel =1/64. , 
                reorient=True, threshold=0.8, scaled=True  )

In [ ]:
STL_fnames=[]

for i in range (9000,10000):
         STL_fnames.append (f'_voxel_0_tester_{i}.stl')
smean_list=[]
svar_list=[]
stddev_list=[]
max_stress_list=[]
modulus_list=[]
voxellist=[]
pointcloudlist=[]
STLfilename=[]
 

In [ ]:
print (STL_fnames)

In [ ]:
smean_list,svar_list,stddev_list,max_stress_list,modulus_list,voxellist,pointcloudlist,STLfilename=run_sims (size_voxel_set=64,
             threshold=0.9,
             sizevoxel =1.,
             k_spring= 5.,
             r0=3.5,
             use_omp = False ,
              loc='./STL_collection/',
              STL_fnames=STL_fnames,
            startjj=9000,                                                                                                 
             smean_list=smean_list,
            svar_list=svar_list,
            stddev_list=stddev_list,
            max_stress_list=max_stress_list,
            modulus_list=modulus_list,
            voxellist=voxellist,
            pointcloudlist=pointcloudlist,
            STLfilename=STLfilename,       
             csv_fname="output_results_9000.csv" ,
          LAMMPS_OUT='./LAMMPS_OUT_9000/',
          pullsteps=100000,
          vspeed=(0, 0, 0.00025),                                                                                       
        coord_name='./data_harmonic_9000.coor',
       show_on=False,
     reorient=True,
     scaled=False,
     fnameout="in_python_FCC_voxel_harmonic_pull_PROCESSED_9000.txt")

In [ ]:

modulus, max_stress=analyze_plot_stress (fname=f'{LAMMPS_OUT}/sscurve.dat', miniter= 0, maxiter=10 , modulus_dir=3 )

In [ ]:
def plot_dist_data (results, ID='modulus', xl=0, xh=500, bins=100, withline=False, plot_raw=False, norm=True):
    modulusdata = results[ID].values

    if plot_raw:
        plt.plot (modulusdata,  'bo-',label=f'{ID} data')
        plt.legend()
        plt.show()

    nf=1    
    if norm:
        nf=statistics.pstdev(modulusdata)
        modulusdata=modulusdata/nf
        print ("Data NORMALIZED BY Standart Dev: ", nf)
        
    if withline==False:
        
        fig_handle = sns.distplot(modulusdata,bins=bins,kde=False,  
                                      rug=False,norm_hist=False,axlabel= ID )
        fig = fig_handle.get_figure()

        fig_handle.set_xlim(xl, xh)

        fig.savefig(f'modulus_distribution_{ID}.png',dpi=200)


        plt.show()
    else:
        fig_handle=sns.histplot(x=modulusdata,    discrete=True,stat='density',
                 color='darkblue', edgecolor='blue', 
                 kde=True, kde_kws={'cut': 5}, line_kws={'linewidth': 2}, bins=bins,)
        fig_handle.set_xlim(xl, xh)
        fig_handle.set_xlabel(ID)#, fontsize = 8)
        fig_handle.set_ylabel("Density")#, fontsize = 20)
        fig.savefig(f'modulus_distribution_fit_{ID}.png',dpi=200)

        plt.show()
        
    d_array = np.array(modulusdata)
    index = np.argmax(d_array)
    trueindex=re.sub(r"\D", "", results['voxel_file'][index])
    fname=results['voxel_file'][index]
    
    index_l = np.argmin(d_array)
    trueindex_l=re.sub(r"\D", "", results['voxel_file'][index_l])
    fname_l=results['voxel_file'][index_l]
    
    print(f"Highest {ID} value at: {index} = {d_array[index]} : {results['voxel_file'][index]}, True index={trueindex}" )
    #print (str(results.columns.values))
    print (f"{results['modulus'][index]} {results['max_stress'][index]} {results['stress_mean'][index]}\
    {results['s_stddev'][index]} {results['s_var'][index]}: {results['voxel_file'][index]}")

 
    
    print(f"Lowest {ID} value at: {index_l} = {d_array[index_l]} : {results['voxel_file'][index_l]}, True index={trueindex_l}" )
    #print (results.columns.values)
    print (f"{results['modulus'][index_l]} {results['max_stress'][index_l]} {results['stress_mean'][index_l]}\
    {results['s_stddev'][index_l]} {results['s_var'][index_l]}: {results['voxel_file'][index_l]}")

    return index, d_array[index], index_l, d_array[index_l], fname, fname_l


In [ ]:
file_path=f'{LAMMPS_OUT}/output_results_averaged.csv'#'pdb_format_update_4_30_picked_318.dat'
results=pd.read_csv(file_path)

results.drop(results[results['modulus'] < 0.1].index, inplace = True)
results.drop(results[results['max_stress'] < 0.1 ].index, inplace = True)

results=results.reset_index(drop=True)



In [ ]:
_,_,_,_,fname, fname_l=plot_dist_data (results,ID='modulus',xl=0, xh=7, bins=100, withline=False, plot_raw=False ,norm=True)
_,_,_,_,fname, fname_l=plot_dist_data (results,ID='max_stress',xl=0, xh=8, bins=100,withline=False, plot_raw=False,norm=True)
_,_,_,_,fname, fname_l=plot_dist_data (results,ID='stress_mean',xl=0, xh= 3, bins=500,withline=False, plot_raw=False,norm=True)
_,_,_,_,fname, fname_l=plot_dist_data (results,ID='s_stddev',xl=0, xh= 2, bins=700,withline=False, plot_raw=False,norm=True)
_,_,_,_,fname, fname_l=plot_dist_data (results,ID='s_var',xl=0, xh= .2, bins=5000,withline=False, plot_raw=False,norm=True)